In [1]:
import pandas as pd
import glob

In [47]:
csvfiles = glob.glob("data/*.xlsx")

dfs = []

for file in csvfiles:
    df = pd.read_excel(file)
    
    dfs.append(df)
    
data = pd.concat(dfs)

data.head(5)

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,2018-01-01,00:29,Monday,SHEPPARD WEST STATION,MUATC,10,15,N,YU,5986
1,2018-01-01,01:07,Monday,DUNDAS STATION,MUNCA,0,0,NaN,YU,0
2,2018-01-01,01:22,Monday,MUSEUM STATION,MUSC,0,0,N,YU,5751
3,2018-01-01,01:28,Monday,BAY LOWER,EUOE,0,0,NaN,BD,5222
4,2018-01-01,01:39,Monday,MUSEUM STATION,MUO,6,11,S,YU,5781


In [46]:
delay_codes = pd.read_csv("delay_codes.csv", index_col="SUB RMENU CODE")

delay_codes.head(2)

,CODE DESCRIPTION
SUB RMENU CODE,
EUAC,Air Conditioning
EUAL,Alternating Current


In [20]:
data2 = data.set_index("Code").join(delay_codes)

data2

,Date,Time,Day,Station,Min Delay,Min Gap,Bound,Line,Vehicle,CODE DESCRIPTION
ERAC,2019-01-30,11:04,Wednesday,MCCOWAN STATION,5,10,N,SRT,3017,Air Conditioning
ERAC,2022-02-22,19:15,Tuesday,MCCOWAN STATION,6,12,S,SRT,3024,Air Conditioning
ERAC,2022-03-30,07:21,Wednesday,MCCOWAN STATION,5,10,NaN,SRT,3000,Air Conditioning
ERAC,2022-07-18,06:52,Monday,LAWRENCE EAST STATION,46,51,N,SRT,3025,Air Conditioning
ERAC,2022-09-07,17:03,Wednesday,MCCOWAN STATION,5,10,N,SRT,3005,Air Conditioning
...,...,...,...,...,...,...,...,...,...,...
TUSUP,2017-05-04,08:50,Thursday,ST CLAIR WEST STATION,6,8,S,YU,5426,Supervisory Error
TUSUP,2017-05-19,21:30,Friday,ST GEORGE YUS STATION,0,0,S,YU,5801,Supervisory Error
TUSUP,2017-05-26,12:45,Friday,NORTH YORK CTR STATION,0,0,N,YU,5551,Supervisory Error
TUTD,2014-04-30,07:22,Wednesday,ISLINGTON STATION,3,6,W,BD,5196,NaN


In [23]:
len(data2.loc[data2["CODE DESCRIPTION"].isna()])

2095

In [22]:
data2.loc[data2["CODE DESCRIPTION"].isna()].index.unique()

Index(['ERTO', 'ETCA', 'EUTAC', 'MFO', 'MTAFR', 'MTO', 'MTUI', 'MUCSA',
       'MUNCA', 'MUPF', 'PUEO', 'PUEWZ', 'PUSIC', 'PUTN', 'SFAE', 'SFAP',
       'SFDP', 'SFO', 'TRNCA', 'TUNCA', 'TUTD', 'XXXXX'],
      dtype='object')

In [24]:
data2 = data2.reset_index()

In [27]:
data2["year"] = data2["Date"].dt.year

In [42]:
reasons_pivot_count = data2.pivot_table(columns="year", index="CODE DESCRIPTION", values="Min Delay", aggfunc="count").sort_values(2021, ascending=False).dropna(subset=[2022]).head(10)

reasons_pivot_count

year,2014,2015,2016,2017,2018,2019,2020,2021,2022
CODE DESCRIPTION,,,,,,,,,
Disorderly Patron,672.0,625.0,800.0,356.0,95.0,109.0,122.0,195.0,1940.0
Injured or ill Customer (In Station) - Transported,518.0,1159.0,1341.0,529.0,125.0,146.0,112.0,113.0,1952.0
Miscellaneous Other,980.0,1059.0,913.0,286.0,64.0,63.0,70.0,71.0,1262.0
Passenger Assistance Alarm Activated - No Trouble Found,1410.0,1315.0,1272.0,460.0,106.0,113.0,108.0,68.0,1446.0
Passenger Other,298.0,319.0,300.0,110.0,31.0,29.0,42.0,48.0,1022.0
ATC Project,NaN,NaN,NaN,NaN,26.0,35.0,37.0,45.0,845.0
Miscellaneous Speed Control,2626.0,4215.0,4415.0,1520.0,283.0,246.0,71.0,42.0,536.0
Injured or ill Customer (In Station) - Medical Aid Refused,112.0,228.0,301.0,110.0,24.0,28.0,34.0,40.0,885.0
Operator Violated Signal,451.0,474.0,485.0,180.0,56.0,33.0,38.0,37.0,365.0


In [45]:
reasons_pivot_mins = data2.pivot_table(columns="year", index="CODE DESCRIPTION", values="Min Delay", aggfunc="sum").sort_values(2021, ascending=False).dropna(subset=[2022]).head(10)

reasons_pivot_mins

year,2014,2015,2016,2017,2018,2019,2020,2021,2022
CODE DESCRIPTION,,,,,,,,,
Priority One - Train in Contact With Person,1932.0,1123.0,1601.0,790.0,104.0,229.0,371.0,792.0,3922.0
Disorderly Patron,2366.0,1996.0,2695.0,985.0,406.0,293.0,482.0,542.0,6446.0
Unauthorized at Track Level,1046.0,1388.0,1496.0,829.0,121.0,53.0,50.0,468.0,5798.0
Injured or ill Customer (On Train) - Transported,2900.0,2633.0,3485.0,1023.0,392.0,528.0,333.0,247.0,2639.0
ATC Project,NaN,NaN,NaN,NaN,211.0,141.0,123.0,191.0,3831.0
Injured or ill Customer (On Train) - Medical Aid Refused,1991.0,1776.0,2217.0,810.0,231.0,289.0,211.0,165.0,2299.0
Passenger Other,1000.0,1150.0,1118.0,416.0,112.0,123.0,112.0,149.0,4132.0
Door Problems - Faulty Equipment,1861.0,1736.0,1603.0,612.0,267.0,180.0,101.0,103.0,1058.0
Misc. Transportation Other - Employee Non-Chargeable,775.0,567.0,770.0,247.0,75.0,81.0,97.0,102.0,993.0


In [57]:
stations_pivot = data2.pivot_table(index="Station", columns="year", values="Min Delay", aggfunc="sum").sort_values(2022, ascending=False)

stations_pivot

year,2014,2015,2016,2017,2018,2019,2020,2021,2022
Station,,,,,,,,,
FINCH STATION,1262.0,1095.0,958.0,495.0,70.0,122.0,151.0,77.0,3067.0
EGLINTON STATION,1823.0,943.0,1107.0,389.0,129.0,103.0,33.0,121.0,2619.0
KENNEDY BD STATION,1232.0,1721.0,1532.0,412.0,112.0,111.0,126.0,104.0,2026.0
KENNEDY SRT STATION,732.0,1663.0,1068.0,510.0,182.0,66.0,17.0,70.0,1995.0
ST GEORGE YU STATION,857.0,624.0,754.0,292.0,58.0,67.0,30.0,53.0,1934.0
...,...,...,...,...,...,...,...,...,...
YORK MILLS STATION (AP,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
YORK MILLS STATION (LE,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
YORKDALE STATION (EXIT,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [43]:
reasons_pivot.sum()

year
2014     7193.0
2015     9559.0
2016     9989.0
2017     3637.0
2018      819.0
2019      816.0
2020      646.0
2021      691.0
2022    10831.0
dtype: float64

In [48]:
data2.loc[data2["Station"].str.contains("GEORGE"), "Station"].unique()

array(['ST GEORGE BD STATION', 'ST GEORGE YUS STATION',
       'UNION TO ST GEORGE', 'ST GEORGE YU STATION',
       'ST GEORGE STATION TO B', 'ST GEORGE TO BROADVIEW',
       'BROADVIEW TO ST GEORGE', 'ST GEORGE STATION',
       'ST GEORGE TO LAWRENCE', 'PAPE TO ST GEORGE',
       'CHRISTIE - ST GEORGE S', 'ST GEORGE STN',
       'ST GEORGE BD STATION -', 'ST. GEORGE STATION'], dtype=object)

In [49]:
data2["Station"] = (data2["Station"]
                    .str.replace("ST GEORGE YUS STATION", "ST GEORGE YU STATION")
                    .str.replace("ST GEORGE STN|ST. GEORGE STATION", "ST GEORGE STATION")
                    )

data2.loc[data2["Station"].str.contains("GEORGE"), "Station"].unique()

C:\Users\dexmc\AppData\Local\Temp/ipykernel_19028/3128147751.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data2["Station"] = (data2["Station"]


array(['ST GEORGE BD STATION', 'ST GEORGE YU STATION',
       'UNION TO ST GEORGE', 'ST GEORGE STATION TO B',
       'ST GEORGE TO BROADVIEW', 'BROADVIEW TO ST GEORGE',
       'ST GEORGE STATION', 'ST GEORGE TO LAWRENCE', 'PAPE TO ST GEORGE',
       'CHRISTIE - ST GEORGE S', 'ST GEORGE BD STATION -'], dtype=object)

### By vehicle

In [ ]:
vehicle_trips = 

In [7]:
data.groupby("Vehicle")["Min Delay"].sum().to_frame().sort_values("Min Delay", ascending=False)

,Min Delay
Vehicle,
0,5406
3007,2155
3002,1421
3012,1343
3015,1280
...,...
4417,0
5965,0
5964,0
